In [ ]:
#  #########################################################################
#  #                           IN THE NAME OF ALLAH                        #
#  #     STEEL GABLE FRAME OPTIMIZATION ANALYSIS WEB HEIGHT NONPRISMATIC   #
#  #              I SECTION COLUMN WITH FINITE PRISMATIC COLUMN            #
#  #                    WITH NONLINEAR PUSHOVER ANALYSIS                   #
#  #                         NEWTON-RAPHSON METHOD                         #
#  #    MODELING OF NONPRISMATIC ELEMENT WITH MULTI PRISMATIC ELEMENTS     #
#  #-----------------------------------------------------------------------#
#  #              THIS PROGRAM WRITTEN BY SALAR DELAVAR QASHQAI            #
#  #                   EMAIL: salar.d.ghashghaei@gmail.com                 #
#  #########################################################################

In [ ]:
#import the os module
import os
import math
import time

In [ ]:
#to create a directory at specified path with name "Data"
os.mkdir('C:\\OPENSEESPY_SALAR')
#this will create the directory with name 'Data' and will update it when we rerun the analysis,
# otherwise we have to keep deleting the old 'Data' Folder
dir = "C:\\OPENSEESPY_SALAR\\OPENSEESPY_DATA"
if not os.path.exists(dir):
    os.makedirs(dir)

In [ ]:
def MAXABS_FUN(DATA_FILE):
    import numpy as np
    # Read and process displacement data
    NameFiles = DATA_FILE
    filename = f"{NameFiles}.txt"
    D = np.loadtxt(filename)
    #print(D)
    MAXABS = np.max(np.abs([D[:, 3]]))
    #print("MAX. ABS. :", MAXABS)
    return MAXABS
# -----------------------------------------------
def PLOT_2D(X, Y, Xfit, Yfit, XLABEL, YLABEL, TITLE, COLOR, Z):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(12, 8))
    if Z == 1:
        # Plot 1 line
        plt.plot(X, Y,color=COLOR)
        plt.xlabel(XLABEL)
        plt.ylabel(YLABEL)
        plt.title(TITLE)
        plt.grid(True)
        plt.show()
    if Z == 2:
        plt.plot(X, Y, Xfit, Yfit, 'r--', linewidth=3)
        plt.title(TITLE)
        plt.xlabel(XLABEL)
        plt.ylabel(YLABEL)
        plt.legend(['curve', 'bilinear fitted'], loc='lower right')
        plt.grid(True)
        plt.show()
# -----------------------------------------------
def OUTPUT_SECOND_COLUMN(X, COLUMN):
    import numpy as np
    # Time History
    filename = f"C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\\{X}.txt"
    data_collected = np.loadtxt(filename)
    X = data_collected[:, COLUMN]
    return X 
# -----------------------------------------------
def BILNEAR_CURVE(Cur, Mom):
    import numpy as np
    # bilinear fitting
    SIZE = len(Mom)
    hh = np.zeros(SIZE-1)
    Aa = np.zeros(SIZE-1)
    for i in range(SIZE-1):
        hh[i] = Cur[i+1] - Cur[i]
        Aa[i] = (Mom[i] + Mom[i+1]) * 0.5 * hh[i]

    Area = sum(Aa)
    k0 = Mom[2] / Cur[2]
    fiy = (Mom[i+1] * max(Cur) * 0.5 - Area) / (Mom[i+1] * 0.5 - k0 * max(Cur) * 0.5)
    My = k0 * fiy
    X = np.array([0, fiy, max(Cur)])
    Y = np.array([0, My, Mom[i+1]])
    
    print('+==========================+')
    print('=   Analysis curve fitted =')
    print('     Disp       Baser Shear')
    print('----------------------------')
    print(np.column_stack((X.T, Y.T)))
    print('+==========================+')
    
    # EI and Ductility_Rito of Unconfined Section
    Elastic_ST = Y[1] / X[1]
    Plastic_ST = Y[2] / X[2]
    Tangent_ST = (Y[2] - Y[1]) / (X[2] - X[1])
    Ductility_Rito = X[2] / X[1]
    Over_Strength_Factor = Y[2] / Y[1]
    
    print('+----------------------------------------------------+')
    print(f' Elastic Stiffness :             {Elastic_ST:.2f}')
    print(f' Plastic Stiffness :             {Plastic_ST:.2f}')
    print(f' Tangent Stiffness :             {Tangent_ST:.2f}')
    print(f' Section Ductility Ratio :       {Ductility_Rito:.2f}')
    print(f' Section Over Strength Factor:   {Over_Strength_Factor:.2f}')
    print('+----------------------------------------------------+')
    
    return X, Y, Elastic_ST, Plastic_ST, Tangent_ST, Ductility_Rito, Over_Strength_Factor

In [ ]:
# pip install openseespy

In [ ]:
# OUTPUT DATA ADDRESS:
SALAR_DIR = 'C:/OPENSEESPY_SALAR/OPENSEESPY_DATA/';

In [ ]:
#
#    ^Y
#    |
#    2       __ 
#    |          | 
#    |          |
#    |          |
#  (1)       LCol
#    |          |
#    |          |
#    |          |
#  =1=      _|_  -------->X
#

# SET UP ----------------------------------------------------------------------------


def PUSHOVER_ANALYSIS(hw_COL, tw_COL,tf_COL, bf_COL, hw_BEAM, tw_BEAM, tf_BEAM, bf_BEAM, H1, H2, L1, N, ND, Weight, UL, DMAX):
    NN = int(N / 4) # 4 ELEMENTS
    import openseespy.opensees as op
    op.wipe()
    op.model('basic', '-ndm', 2, '-ndf', 3) 
    PCol =Weight  # nodal dead-load weight per column
    g =  9810 # mm/s^2
    Mass =  PCol/g
    h1 = H1 / (NN - 1)
    h2 = H2 / (NN - 1)
    bl = (L1 **2 + H2 **2)**0.5  # Beam Length
    l1 = L1 / (NN) 
          
    IDctrlNode = ND ## INCREMENTAL DISPLACEMENT NODE
    IDctrlDOF = 1
    
    # MATERIAL parameters -------------------------------------------------------------------
    IDreinf = 1; 				# material ID tag -- steel 
    Fy = 240			# STEEL yield stress
    Cy = 0.0012			# STEEL yield stress
    Es = Fy/Cy				# modulus of steel
    Bs = 0.01				# strain-hardening ratio 
    R0 = 18.0				# control the transition from elastic to plastic branches
    cR1 = 0.925				# control the transition from elastic to plastic branches
    cR2 = 0.15				# control the transition from elastic to plastic branches
    op.uniaxialMaterial('Steel02', IDreinf, Fy, Es, Bs, R0,cR1,cR2) # build reinforcement material
    # ---------------------------------------------------------------------------------------
    hwbot_COL = 0.5 * hw_COL # Bottom column web height
    # nodal coordinates:
    op.node(1, 0.0, 0.0) # node#, X, Y
    op.node(N, L1 * 2, 0.0) # node#, X, Y
    # Single point constraints -- Boundary Conditions
    op.fix(1, 1, 1, 1) # node DX DY RZ
    op.fix(N, 1, 1, 1) # node DX DY RZ
    
    # ----------------------------------------------------------
    # COLUMN 01:
    hwbot_COL = 0.5 * hw_COL # Bottom column web height - SECTION 01
    for i in range(2, NN + 1, 1):
        pp = i-1
        x = 0; y = h1 * pp;
        op.node(i, x, y)
        ColSecTag = pp			# assign a tag number to the column section
        ColTransfTag = pp
        eleTag = pp

        # FIBER SECTION properties -------------------------------------------------------------
        # symmetric section
        #                        y
        #                        ^
        #                        |     
        #              _____________________    --   --
        #             |_________   ________|    |    -- tf
        #                      |  |             |
        #                      |  |             |
        #    z <---       hw   |tw|             H
        #                      |  |             |
        #              ________|  |_________    |
        #             |____________________|    |    -- tf
        #                                      --    --
        #             |-------- bf --------|
        #
        # STEEL I SECTION: 
        HW_COL =  hwbot_COL + ((hw_COL - hwbot_COL) / H1) * h1 * (i-1) # Varying web height of nonprismatic section - SECTION 02
        #print(i, x, y, i-1, i, HW_COL)
        coverY = (HW_COL + tf_COL) / 2.0
        coverZ = tw_COL / 2.0
        coreY = coverY - tf_COL
        coreZ02 = bf_COL / 2.0

        nfCoreY = 15;			# number of fibers for steel in y-direction
        nfCoreZ = 5;			# number of fibers for steel in z-direction


        op.section('Fiber', ColSecTag)
        # Define the core patch
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coverY, coreZ02, -coverY, -coreZ02, -coreY,-coreZ02, coreY, coreZ02) # TOP FLANGE
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coreY,coverZ, -coreY,-coverZ, coreY,-coverZ, coreY, coverZ) # MIDDLW WEB
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, coreY, coreZ02, -coreY, coreZ02, coverY,-coreZ02, coreY, coreZ02) # TOP FLANGE

        op.geomTransf('Linear', ColTransfTag)
        numIntgrPts = 5
        op.element('nonlinearBeamColumn', eleTag, i-1, i, numIntgrPts, ColSecTag, 1)
    # ----------------------------------------------------------
    # BEAM 01:
    hwbot_BEAM = 0.5 * hw_BEAM # j section beam web height  - SECTION 04
    for i in range(NN + 1, 2 * NN + 1, 1):
        zz = i - NN
        x = l1 * zz; y = H1 + h2 * zz;
        op.node(i, x, y)
        ColSecTag = i-1			# assign a tag number to the column section
        ColTransfTag = i-1
        eleTag = i-1

        # FIBER SECTION properties -------------------------------------------------------------
        # symmetric section
        #                        y
        #                        ^
        #                        |     
        #              _____________________    --   --
        #             |_________   ________|    |    -- tf
        #                      |  |             |
        #                      |  |             |
        #    z <---       hw   |tw|             H
        #                      |  |             |
        #              ________|  |_________    |
        #             |____________________|    |    -- tf
        #                                      --    --
        #             |-------- bf --------|
        #
        # STEEL I SECTION: 
        HW_BEAM =  hw_BEAM + ((hwbot_BEAM - hw_BEAM) / L1) * l1 * zz # Varying web height of nonprismatic section - SECTION 03
        #print(i, x, y, i-1, i, HW_BEAM)
        coverY = (HW_BEAM + tf_BEAM) / 2.0
        coverZ = tw_BEAM / 2.0
        coreY = coverY - tf_BEAM
        coreZ02 = bf_BEAM / 2.0

        nfCoreY = 15;			# number of fibers for steel in y-direction
        nfCoreZ = 5;			# number of fibers for steel in z-direction


        op.section('Fiber', ColSecTag)
        # Define the core patch
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coverY, coreZ02, -coverY, -coreZ02, -coreY,-coreZ02, coreY, coreZ02) # TOP FLANGE
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coreY,coverZ, -coreY,-coverZ, coreY,-coverZ, coreY, coverZ) # MIDDLW WEB
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, coreY, coreZ02, -coreY, coreZ02, coverY,-coreZ02, coreY, coreZ02) # TOP FLANGE

        op.geomTransf('Linear', ColTransfTag)
        numIntgrPts = 5
        op.element('nonlinearBeamColumn', eleTag, i-1, i, numIntgrPts, ColSecTag, 1) 
    # ----------------------------------------------------------
    # COLUMN 02:
    hwbot_COL = 0.5 * hw_COL # Bottom column web height
    for i in range(N-1, 3 * NN, -1):
        pp = i - 1
        zz = N - i
        #print(pp)
        x = L1 * 2; y = h1 * zz;
        #print(x, y)
        op.node(i, x, y)
        ColSecTag = i			# assign a tag number to the column section
        ColTransfTag = i
        eleTag = i

        # FIBER SECTION properties -------------------------------------------------------------
        # symmetric section
        #                        y
        #                        ^
        #                        |     
        #              _____________________    --   --
        #             |_________   ________|    |    -- tf
        #                      |  |             |
        #                      |  |             |
        #    z <---       hw   |tw|             H
        #                      |  |             |
        #              ________|  |_________    |
        #             |____________________|    |    -- tf
        #                                      --    --
        #             |-------- bf --------|
        #
        # STEEL I SECTION: 
        HW_COL =  hwbot_COL + ((hw_COL - hwbot_COL) / H1) * h1 * (N - i) # Varying web height of nonprismatic section
        #print(i, x, y, i+1, i, HW_COL)
        coverY = (HW_COL + tf_COL) / 2.0
        coverZ = tw_COL / 2.0
        coreY = coverY - tf_COL
        coreZ02 = bf_COL / 2.0

        nfCoreY = 15;			# number of fibers for steel in y-direction
        nfCoreZ = 5;			# number of fibers for steel in z-direction


        op.section('Fiber', ColSecTag)
        # Define the core patch
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coverY, coreZ02, -coverY, -coreZ02, -coreY,-coreZ02, coreY, coreZ02) # TOP FLANGE
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coreY,coverZ, -coreY,-coverZ, coreY,-coverZ, coreY, coverZ) # MIDDLW WEB
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, coreY, coreZ02, -coreY, coreZ02, coverY,-coreZ02, coreY, coreZ02) # TOP FLANGE

        op.geomTransf('Linear', ColTransfTag)
        numIntgrPts = 5
        op.element('nonlinearBeamColumn', eleTag, i+1, i, numIntgrPts, ColSecTag, 1)  
    # ----------------------------------------------------------
    # BEAM 02:
    hwbot_BEAM = 0.5 * hw_BEAM # j section beam web height
    for i in range(3 * NN, 2 * NN, -1):
        zz = 3 * NN + 1 - i
        x = 2 * L1 - l1 * zz; y = H1 + h2 * zz;
        op.node(i, x, y) 
        ColSecTag = i			# assign a tag number to the column section
        ColTransfTag = i
        eleTag = i


        # FIBER SECTION properties -------------------------------------------------------------
        # symmetric section
        #                        y
        #                        ^
        #                        |     
        #              _____________________    --   --
        #             |_________   ________|    |    -- tf
        #                      |  |             |
        #                      |  |             |
        #    z <---       hw   |tw|             H
        #                      |  |             |
        #              ________|  |_________    |
        #             |____________________|    |    -- tf
        #                                      --    --
        #             |-------- bf --------|
        #
        # STEEL I SECTION: 
        HW_BEAM =  hw_BEAM + ((hwbot_BEAM - hw_BEAM) / L1) * l1 * zz # Varying web height of nonprismatic section
        #print(i, x, y, i+1, i, HW_BEAM)
        coverY = (HW_BEAM + tf_BEAM) / 2.0
        coverZ = tw_BEAM / 2.0
        coreY = coverY - tf_BEAM
        coreZ02 = bf_BEAM / 2.0

        nfCoreY = 15;			# number of fibers for steel in y-direction
        nfCoreZ = 5;			# number of fibers for steel in z-direction

        op.section('Fiber', ColSecTag)
        # Define the core patch
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coverY, coreZ02, -coverY, -coreZ02, -coreY,-coreZ02, coreY, coreZ02) # TOP FLANGE
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coreY,coverZ, -coreY,-coverZ, coreY,-coverZ, coreY, coverZ) # MIDDLW WEB
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, coreY, coreZ02, -coreY, coreZ02, coverY,-coreZ02, coreY, coreZ02) # TOP FLANGE

        op.geomTransf('Linear', ColTransfTag)
        numIntgrPts = 5
        op.element('nonlinearBeamColumn', eleTag, i+1, i, numIntgrPts, ColSecTag, 1)  
        
        
    op.element('nonlinearBeamColumn', 200, 102, 100, numIntgrPts, ColSecTag, 1) ## THIS ELEMENT IS FOR CONNECTION 2 SEPARATE ELEMENTS TO EACH OTHER  
    # Ouput Displacement and Base-Shear
    op.recorder('Node', '-file', f"{SALAR_DIR}DTH.txt",'-time', '-node', ND, '-dof', 1,2,3, 'disp')# Displacement Time History
    op.recorder('Node', '-file', f"{SALAR_DIR}BTH.txt",'-time', '-node', 1, '-dof', 1,2,3, 'reaction')# Base Shear Time History

    # node#, Mx My Mz, Mass=Weight/g, neglect rotational inertia at nodes
    #op.mass(2*NN, Mass, 1e-9, 0.0)
    #defining gravity loads
    op.timeSeries('Linear', 1)
    op.pattern('Plain', 1, 1)
    for i in range(NN + 1, 2 * NN + 1, 1): # CREATE UNIFORM LOADS FOR BEAM 01
        op.eleLoad('-ele', i-1,'-type', '-beamUniform', UL, 0.0) # uniformly-distributed load
    for i in range(3 * NN, 2 * NN, -1):# CREATE UNIFORM LOADS FOR BEAM 02
        op.eleLoad('-ele', i,'-type', '-beamUniform', UL, 0.0) # uniformly-distributed load
    
    Tol = 1e-8 # convergence tolerance for test
    Iter = 1000# convergence iteration for test
    NstepGravity = 10
    DGravity = 1 / NstepGravity
    op.integrator('LoadControl', DGravity) # determine the next time step for an analysis
    op.numberer('Plain') # renumber dof's to minimize band-width (optimization), if you want to
    op.system('BandGeneral') # how to store and solve the system of equations in the analysis
    op.constraints('Plain') # how it handles boundary conditions
    op.test('NormDispIncr', Tol, Iter) # determine if convergence has been achieved at the end of an iteration step
    op.algorithm('Newton') # use Newton's solution algorithm: updates tangent stiffness at every iteration
    op.analysis('Static') # define type of analysis static or transient
    op.analyze(NstepGravity) # apply gravity

    op.loadConst('-time', 0.0) #maintain constant gravity loads and reset time to zero
    print('Model Built')
    
    Dincr = 0.001 * DMAX
    Hload = 1#Weight
    maxNumIter = 1000
    tol = 1e-8

    op.timeSeries('Linear', 2)
    op.pattern('Plain', 200, 2)
    op.load(ND, Hload, 0.0, 0.0)

    op.wipeAnalysis()
    op.constraints('Plain')
    op.numberer('Plain')
    op.system('BandGeneral')
    op.test('EnergyIncr', Tol, maxNumIter)
    op.algorithm('Newton')

    op.integrator('DisplacementControl', IDctrlNode, IDctrlDOF, Dincr)
    op.analysis('Static')


    Nsteps =  int(DMAX/ Dincr)

    ok = op.analyze(Nsteps)
    #print(ok)

    # for gravity analysis, load control is fine, 0.1 is the load factor increment (http://opensees.berkeley.edu/wiki/index.php/Load_Control)

    test = {1:'NormDispIncr', 2: 'RelativeEnergyIncr', 4: 'RelativeNormUnbalance',5: 'RelativeNormDispIncr', 6: 'NormUnbalance'}
    algorithm = {1:'KrylovNewton', 2: 'SecantNewton' , 4: 'RaphsonNewton',5: 'PeriodicNewton', 6: 'BFGS', 7: 'Broyden', 8: 'NewtonLineSearch'}

    for i in test:
        for j in algorithm:

            if ok != 0:
                if j < 4:
                    op.algorithm(algorithm[j], '-initial')

                else:
                    op.algorithm(algorithm[j])

                op.test(test[i], Tol, 1000)
                ok = op.analyze(Nsteps)                            
                #print(test[i], algorithm[j], ok)             
                if ok == 0:
                    break
            else:
                continue

    #u2 = op.nodeDisp(2, 1)
    #print("u2 = ", u2)
    print('Pushover Done')
    op.wipe()
    


In [ ]:
### --------------------------------------------------------------
###          NONPRISMATIC I SECTION WEB HEIGHT OPTIMIZATION
### --------------------------------------------------------------

# define section geometry
H1 = 3000.0 # [mm] Column length
H2 = 1000.0 # [mm] Beam length
L1 = 5000.0 # [mm] Beam length

hw_COL = 350 # [mm] Section Web Height 
tw_COL = 10 # [mm] Section Web Thickness
tf_COL = 10 # [mm] Section Flange Thickness
bf_COL = 110 # [mm] Section Flange Width

hw_BEAM = 150 # [mm] Section Web Hight 
tw_BEAM = 10 # [mm] Section Web Thickness
tf_BEAM = 10 # [mm] Section Flange Thickness
bf_BEAM = 110 # [mm] Section Flange Width

N = 200 # STRUCTURES NODES COUNT
ND = 150 # NODE NUMBER APPLIED INCREMENTAL DISPLACEMENT

Weight = 100000.0 # [N] superstructure weight
ul = -10 #[N/mm] Uniform Distributed Loads

X = hw_COL # Intial Guess for I Section web height
ESP = 1e-3 # Finite difference derivative Convergence Tolerance
TOLERANCE = 1e-4 # Convergence Tolerance
RESIDUAL = 100 # Convergence Residual 
IT = 0 # Intial Iteration
ITMAX = 100000 # Max. Iteration
DMAX = 300 # [mm] Max. Pushover Incremental Displacement

TARGET_BASEMOMENT = 2.5e8 # [N] Target Demand Max. Abs. Base Moment
DATA_FILE ='C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\BTH'  # MAX BASE MOMENT

###########################################################################
#         NOTICE:                                                         #
#   FOR BETTER CONVERGENCE FIRST VALUE MAXABS_FUN(DATA_FILE) IN ANALYSIS  #
#   CYCLE  RECOMMEND TO BE LESS THAN TARGET_BASEMOMENT.                   #
###########################################################################

#TARGET_DISP = 7 # [mm] Target Demand Max. Abs. Displacement
#DATA_FILE ='C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\DTH'  # MAX DISPLACEMENT

# monitor cpu time
import time
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(f"Current time (HH:MM:SS): {current_time}\n\n")

### FIND THE OPTIMUM VALUE 
while (RESIDUAL > TOLERANCE):
    PUSHOVER_ANALYSIS(X, tw_COL,tf_COL, bf_COL, hw_BEAM, tw_BEAM, tf_BEAM, bf_BEAM, H1, H2, L1, N, ND, Weight, ul, DMAX)
    time.sleep(10);# Sleep for 10 seconds
    F = MAXABS_FUN(DATA_FILE) - TARGET_BASEMOMENT
    print('Current Max. Abs. Base Moment: ', MAXABS_FUN(DATA_FILE))
    #print('F: ', F)
    # Evaluate at Xmain and Fmin
    Xmin = X - ESP
    PUSHOVER_ANALYSIS(Xmin, tw_COL,tf_COL, bf_COL, hw_BEAM, tw_BEAM, tf_BEAM, bf_BEAM, H1, H2, L1, N, ND, Weight, ul, DMAX)
    time.sleep(10);# Sleep for 10 seconds
    Fmin = MAXABS_FUN(DATA_FILE) - TARGET_BASEMOMENT
    #print('Fmin: ', Fmin)
    # Evaluate at Xmax and Fmax
    Xmax = X + ESP
    PUSHOVER_ANALYSIS(Xmax, tw_COL,tf_COL, bf_COL, hw_BEAM, tw_BEAM, tf_BEAM, bf_BEAM, H1, H2, L1, N, ND, Weight, ul, DMAX)
    time.sleep(10);# Sleep for 10 seconds
    Fmax = MAXABS_FUN(DATA_FILE) - TARGET_BASEMOMENT
    #print('Fmax: ', Fmax)
    DF = (Fmax-Fmin)/(2*ESP);# Calculate the Finite difference derivative of F
    #print('DF: ', DF)
    DX = F / DF; # Calculate dx
    RESIDUAL = abs(DX); # Calculate residual
    print('RESIDUAL: ', RESIDUAL, ' - WEB HEIGHT: ', X)
    X -= DX; # update X
    IT += 1; # update iteration
    if IT == ITMAX:
        print("\t\t Iteration reached to Max. Iteration")
        print("\t\t Change ESP and TOLERANCE for better Convergence")
        X = -1
        break;
    if RESIDUAL < TOLERANCE:
        print(f'\t\t Optimum Section 02 Web Height:   {X:.4f}')
        print(f'\t\t Iteration Counts:                {IT}')
        print(f'\t\t Convergence Residual:            {RESIDUAL:.10e}')
    #print(X)
    
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(f"Current time (HH:MM:SS): {current_time}\n\n")

In [ ]:
#### PLOT COLUMN LENGTH AND WEB HEIGHT NONPRISMATIC I SECTION CLOUMN WITH FINITE PRISMATIC CLOUMN
hw = X # Optimum Web Height
NN = int(N / 4)
z = []
x = []
l = H1 / NN
hwbot = 0.5 * hw
# Top column web height
for i in range(2, NN+1, 1):
    L = l * (i-1)
    x.append(L)
    HW =  hwbot + ((hw - hwbot) / H1) * L # Varying web height of nonprismatic section
    z.append(HW)
    print(f' Element: {i-1} Column Length: {L:.2f} - Web Height: {HW:.2f}')

import matplotlib.pyplot as plt


# Plot shear force and bending moment
plt.figure(figsize=(12, 8))
plt.plot(x, z,color='black')
plt.xlabel('COLUMN LENGTH')
plt.ylabel('WEB HEITHT')
plt.title(f'COLUMN 1 LENGTH AND WEB HEIGHT NONPRISMATIC I SECTION WITH FINITE {N} PRISMATIC SECTIONS')
plt.grid();plt.show();    

In [ ]:
BASE_SHEAR = abs(OUTPUT_SECOND_COLUMN('BTH', 1))
DISP = OUTPUT_SECOND_COLUMN('DTH', 1)
XX, YY, Elastic_ST, Plastic_ST, Tangent_ST, Ductility_Rito, Over_Strength_Factor = BILNEAR_CURVE(DISP, BASE_SHEAR)
PLOT_2D(DISP, BASE_SHEAR, XX, YY, XLABEL='DISPLACEMENT DOF (448)', YLABEL='BASE SHEAR DOF (1)', TITLE='PUSHOVER DISPLACEMENT BASE-SHEAR CURVE', COLOR='blue', Z=2)